# First steps

### API keys

In [ ]:
QDRANT_API_KEY = "voir mail"
MISTRAL_API_KEY = "voir mail"

### Install modules

In [2]:
!pip install mistralai > /dev/null
!pip install llama-index > /dev/null
!pip install llama-index-core > /dev/null
!pip install llama-index-vector-stores-qdrant > /dev/null
!pip install llama-index-postprocessor > /dev/null
!pip install llama-index-query-engine > /dev/null
!pip install llama-index-embeddings-huggingface > /dev/null
!pip install llama-index-llms-ollama > /dev/null
!pip install llama-index-llms-mistralai > /dev/null
!pip install qdrant-client > /dev/null
!pip install llama-index-retrievers > /dev/null
!pip install llama-index-core-retrievers > /dev/null
!pip install llama-index-retrievers-bm25 > /dev/null
!pip install llama-index-core-post-processor > /dev/null
!pip install llama-index-core-schema > /dev/null
!pip install streamlit > /dev/null
!npm install localtunnel > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.9.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
google-cloud-bigtable 2.27.0 requires google-api-core[grpc]<3.0.0dev,>=2.16.0, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-translate 3.12.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.19.5, but you have protobuf 5.29.4 which is incompatible.
pandas-gbq 0.25.0 requires google-api-core<3.0.

### Import modules

In [3]:
import pickle
import gc

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, StorageContext
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.mistralai import MistralAI
from qdrant_client import QdrantClient

from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.postprocessor import SentenceTransformerRerank
import Stemmer
from llama_index.core import VectorStoreIndex, QueryBundle
from llama_index.core.schema import NodeWithScore
from typing import List

### Settings

In [4]:
qdrant_client = QdrantClient(
    "https://3cf7f5eb-3647-41f9-94aa-6dbc76a973d4.us-west-1-0.aws.cloud.qdrant.io:6333",
    port = 6333,
    api_key = QDRANT_API_KEY
)
Settings.embed_model = HuggingFaceEmbedding(model_name="baai/bge-small-en-v1.5")
Settings.llm = MistralAI(api_key=MISTRAL_API_KEY)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Utils

In [15]:
def load_pkl(path):
    with open(path, "rb") as f:
        return pickle.load(f)

def save_pkl(path, obj2save):
    with open(path, "wb") as f:
        pickle.dump(obj2save, f)

# Database

In [24]:
class Qdrant_DB:
    def __init__(self, client: QdrantClient, collection_name="data-battle-v3") -> None:
        self.client = client
        self.collection_name = collection_name
        self.vector_store = self.init_qdrant_vector_store()
        self.vector_index = self.init_vector_index()
        self.retriever = self.init_retriever()
        
    # Init Qdrant vector store
    def init_qdrant_vector_store(self):
        return QdrantVectorStore(client=self.client, collection_name=self.collection_name)

    # create an index
    def init_vector_index(self):
        return VectorStoreIndex.from_vector_store(self.vector_store)
        
    def init_retriever(self, similarity_top_k=2):
        return self.vector_index.as_retriever(similarity_top_k=similarity_top_k)
    
    def add_nodes(self, nodes):
        self.vector_store.upsert(nodes)
        self.vector_index = self.init_vector_index()
        self.retriever = self.init_retriever()
        
    def get_nodes(self):
        return self.vector_store.get_nodes()

    def _retrieve(self, query_bundle: QueryBundle):
        return self.retriever.retrieve(query_bundle)

# Retrievers

In [147]:
class EmbeddingBM25RerankerRetriever(BaseRetriever) :
    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        bm25_retriever: BM25Retriever,
        reranker: SentenceTransformerRerank,
    ) -> None:
        self._vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        self.reranker = reranker

        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        bm25_nodes = self.bm25_retriever.retrieve(query_bundle)

        vector_nodes.extend(bm25_nodes)

        retrieved_nodes = self.reranker.postprocess_nodes(
            vector_nodes, query_bundle
        )

        return retrieved_nodes

# create the Qdrant retriever 
def create_embedding_retriever(qdrant_db: Qdrant_DB, similarity_top_k=50) :
    return qdrant_db.init_retriever(similarity_top_k)

# create the bm25 retriever for a given list of nodes
def create_bm25_retriever(nodes, similarity_top_k=15) :
    return BM25Retriever.from_defaults(
        nodes = nodes,
        similarity_top_k = similarity_top_k,
        stemmer = Stemmer.Stemmer("english"),
        language = "english",
    )

# create the full retriever 
def create_full_retriever(qdrant_db: Qdrant_DB, emb_k=50, bm25_k=15, reranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2", reranker_n=6) :
    # create the embedding retriever 
    embedding_retriever = create_embedding_retriever(qdrant_db, similarity_top_k=emb_k)

    # get nodes for the BM25 retriever
    nodes = qdrant_db.get_nodes()

    # create the BM25 retriever
    bm25_retriever = create_bm25_retriever(
        nodes, similarity_top_k=bm25_k
    )

    # create the reranker
    reranker = SentenceTransformerRerank(
        model=reranker_model, top_n=reranker_n
    )

    # fusion retrievers and add the reranker
    embedding_bm25_retriever_rerank = EmbeddingBM25RerankerRetriever(
        embedding_retriever, bm25_retriever, reranker=reranker
    )
    
    return embedding_bm25_retriever_rerank

# create the retriever based on the retriever_name argument
def create_chosen_retriever(qdrant_db: Qdrant_DB, retriever_name="Embedding/BM25 retrievers + rerank", emb_k=50, bm25_k=15, reranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2", reranker_n=6) :
    # create the embedding retriever 
    if retriever_name == "Embedding retriever":
        return create_embedding_retriever(qdrant_db, similarity_top_k=emb_k)

    # create the BM25 retriever
    elif retriever_name == "BM25 retriever":
        nodes = qdrant_db.get_nodes()
        return create_bm25_retriever(nodes, similarity_top_k=bm25_k)

    # create the full retriever
    else:
        return create_full_retriever(qdrant_db, emb_k=emb_k, bm25_k=bm25_k, reranker_model=reranker_model, reranker_n=reranker_n)

# retrieve context and sources 
def retrieve_context_and_sources(retriever, query, show_scores=False):
    retrieved_nodes = retriever.retrieve(query)
    # concatenate the chunks to have the context 
    context = "\n*********\n".join([node.text for node in retrieved_nodes])

    # get sources from the chunks 
    sources = []
    for node in retrieved_nodes:
        file_name = node.metadata.get("file_name", "Inconnu")  
        page_label = node.metadata.get("page_label")  

        if page_label:
            sources.append(f"{file_name} (Page {page_label})")
        else:
            sources.append(file_name)
        
        if show_scores:
            sources.append(f" - Score: {node.score:.2f}")

    return context, sources

# Example of use 
### *(only retrieve documents, doesn't generate answers)*

In [148]:
from qdrant_client import QdrantClient

client = QdrantClient(
    "https://3cf7f5eb-3647-41f9-94aa-6dbc76a973d4.us-west-1-0.aws.cloud.qdrant.io:6333",
    port = 6333,
    api_key = QDRANT_API_KEY
)

qdrant_db = Qdrant_DB(client)
# full retriever by default
retriever = create_chosen_retriever(qdrant_db)

query = "What are European Patent Application?"

context, sources = retrieve_context_and_sources(retriever, query, show_scores=True)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

# Run Streamlit

# .env file

In [ ]:
%%writefile api_key.py

QDRANT_API_KEY = "voir mail"
MISTRAL_API_KEY = "voir mail"

Writing api_key.py


# database.py file

In [107]:
%%writefile database.py

import pickle

from qdrant_client import QdrantClient

from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, QueryBundle

class Qdrant_DB:
    def __init__(self, client: QdrantClient, collection_name="data-battle-v3") -> None:
        self.client = client
        self.collection_name = collection_name
        self.vector_store = self.init_qdrant_vector_store()
        self.vector_index = self.init_vector_index()
        self.retriever = self.init_retriever()
        
    # Init Qdrant vector store
    def init_qdrant_vector_store(self):
        return QdrantVectorStore(client=self.client, collection_name=self.collection_name)

    # create an index
    def init_vector_index(self):
        return VectorStoreIndex.from_vector_store(self.vector_store)
        
    def init_retriever(self, similarity_top_k=2):
        return self.vector_index.as_retriever(similarity_top_k=similarity_top_k)
    
    def add_nodes(self, nodes):
        self.vector_store.upsert(nodes)
        self.vector_index = self.init_vector_index()
        self.retriever = self.init_retriever()
        
    def get_nodes(self):
        return self.vector_store.get_nodes()

    def _retrieve(self, query_bundle: QueryBundle):
        return self.retriever.retrieve(query_bundle)


Overwriting database.py


# retriever.py file

In [150]:
%%writefile retriever.py

from llama_index.core.retrievers import BaseRetriever, VectorIndexRetriever
from llama_index.retrievers.bm25 import BM25Retriever
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.vector_stores.qdrant import QdrantVectorStore
from llama_index.core import VectorStoreIndex, QueryBundle, StorageContext
from llama_index.core.schema import NodeWithScore
from llama_index.core.query_engine import RetrieverQueryEngine
from typing import List
import Stemmer
import gc

import database

class EmbeddingBM25RerankerRetriever(BaseRetriever) :
    def __init__(
        self,
        vector_retriever: VectorIndexRetriever,
        bm25_retriever: BM25Retriever,
        reranker: SentenceTransformerRerank,
    ) -> None:
        self._vector_retriever = vector_retriever
        self.bm25_retriever = bm25_retriever
        self.reranker = reranker

        super().__init__()

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        vector_nodes = self._vector_retriever.retrieve(query_bundle)
        bm25_nodes = self.bm25_retriever.retrieve(query_bundle)

        vector_nodes.extend(bm25_nodes)

        retrieved_nodes = self.reranker.postprocess_nodes(
            vector_nodes, query_bundle
        )

        return retrieved_nodes

# create the Qdrant retriever 
def create_embedding_retriever(qdrant_db: database.Qdrant_DB, similarity_top_k=50) :
    return qdrant_db.init_retriever(similarity_top_k)

# create the bm25 retriever for a given list of nodes
def create_bm25_retriever(nodes, similarity_top_k=15) :
    return BM25Retriever.from_defaults(
        nodes = nodes,
        similarity_top_k = similarity_top_k,
        stemmer = Stemmer.Stemmer("english"),
        language = "english",
    )

# create the full retriever 
def create_full_retriever(qdrant_db: database.Qdrant_DB, emb_k=50, bm25_k=15, reranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2", reranker_n=6) :
    # create the embedding retriever 
    embedding_retriever = create_embedding_retriever(qdrant_db, similarity_top_k=emb_k)

    # get nodes for the BM25 retriever
    nodes = qdrant_db.get_nodes()

    # create the BM25 retriever
    bm25_retriever = create_bm25_retriever(
        nodes, similarity_top_k=bm25_k
    )

    # create the reranker
    reranker = SentenceTransformerRerank(
        model=reranker_model, top_n=reranker_n
    )

    # fusion retrievers and add the reranker
    embedding_bm25_retriever_rerank = EmbeddingBM25RerankerRetriever(
        embedding_retriever, bm25_retriever, reranker=reranker
    )
    
    return embedding_bm25_retriever_rerank

# create the retriever based on the retriever_name argument
def create_chosen_retriever(qdrant_db: database.Qdrant_DB, retriever_name="Embedding/BM25 retrievers + rerank", emb_k=50, bm25_k=15, reranker_model="cross-encoder/ms-marco-MiniLM-L-6-v2", reranker_n=6) :
    # create the embedding retriever 
    if retriever_name == "Embedding retriever":
        return create_embedding_retriever(qdrant_db, similarity_top_k=emb_k)

    # create the BM25 retriever
    elif retriever_name == "BM25 retriever":
        nodes = qdrant_db.get_nodes()
        return create_bm25_retriever(nodes, similarity_top_k=bm25_k)

    # create the full retriever
    else:
        return create_full_retriever(qdrant_db, emb_k=emb_k, bm25_k=bm25_k, reranker_model=reranker_model, reranker_n=reranker_n)

# retrieve context and sources 
def retrieve_context_and_sources(retriever, query, show_scores=False):
    retrieved_nodes = retriever.retrieve(query)

    # concatenate the chunks to have the context 
    context = "\n*********\n".join([node.text for node in retrieved_nodes])

    # get sources from the chunks 
    sources = []
    for node in retrieved_nodes:
        file_name = node.metadata.get("file_name", "Inconnu")  
        page_label = node.metadata.get("page_label")
        
        if page_label:
            sources.append(f"{file_name} (Page {page_label})")
        else:
            sources.append(file_name)
        if show_scores:
            sources.append(f" - Score: {node.score:.2f}")

    return context, sources

Overwriting retriever.py


# memory.py file

In [109]:
%%writefile memory.py

from llama_index.core import Settings
from llama_index.core.memory import ChatSummaryMemoryBuffer
from llama_index.llms.mistralai import MistralAI

import api_key

# initialisation
from llama_index.llms.mistralai import MistralAI
Settings.llm = MistralAI(api_key=api_key.MISTRAL_API_KEY)

# create memory
memory = ChatSummaryMemoryBuffer.from_defaults(
    chat_history = [],
    llm = Settings.llm,
    token_limit = 300
)
history = memory.get()

# update memory
def update_memory(memory, *args) :
    for new_chat in args :
        memory.put(new_chat)
    return memory, memory.get()

Overwriting memory.py


In [155]:
%%writefile chatbot.py 

import time
import streamlit as st
import requests
from llama_index.core.llms import ChatMessage
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from qdrant_client import QdrantClient
from mistralai import Mistral

import memory
import api_key
import retriever
import database

# chat with Mistral
def chat(client, query):
    response = client.chat.stream(
        model="mistral-large-latest",
        messages=[
            {
                "role": "user",
                "content": query,
            },
        ]
    )
    for chunk in response:
        yield chunk.data.choices[0].delta.content or ""

# merge query, conversation history and Qdrant response
def merge(query, history, qdrant_response, language) :
    history = "\n".join([str(el) for el in history])
    return f"""
Tu es un assistant juridique expert en droit des brevets. Tu aides l'utilisateur à comprendre et à naviguer dans les aspects juridiques liés aux brevets (ex : dépôt, validité, contrefaçon, brevets européens, procédure à l'INPI ou à l'OEB, etc.), notamment dans le cadre de la préparation de concours. 

Tu dois conserver en mémoire tous les éléments. Pour cela, voici l'historique de la conversation ou un résumé détaillé : 
```
{history}
```

L'utilisateur a envoyé le message suivant. C'est à celui-ci, et celui-ci uniquement, que tu vas répondre dans ta prochaine réponse. Le voici :
```
{query}
```

Pour y répondre, tu pourras te baser dans un premier temps sur le texte suivant, issu d'une base de connaissances spécialisée dans le domaine :
```
{qdrant_response}
```

Quelques autres remarques très importantes :
- Tu dois IMPÉRATIVEMENT répondre dans la langue suivante, peu importe la langue dans laquelle l'utilisateur écrit ses messages : {language}. 
- Ta réponse devra être claire, précise, pédagogique, et uniquement en lien avec le droit des brevets. 
- Si le texte fourni (issue de la base de données) n'a rien à voir avec la demande de l'utilisateur, tu pourras dans ce cas y répondre par tes connaissances personnelles, mais tu devras impérativement le préciser de manière explicite à l'utilisateur. 
- Si tu n'as pas la réponse à la question de l'utilisateur, tu lui indiqueras clairement et honnêtement plutôt que de l'inventer. 
- Termine impérativement ton prompt par "**Sources** :", je mettrais les sources en-dessous (tu peux évidemment changer la langue si besoin)
"""

# initialisation
st.title("CYentists' chatbot")
if "session_started" not in st.session_state :
    Settings.embed_model = HuggingFaceEmbedding(model_name="baai/bge-small-en-v1.5")
    st.session_state.mistral_client = Mistral(api_key=api_key.MISTRAL_API_KEY)
    st.session_state.qdrant_client = QdrantClient(
        "https://3cf7f5eb-3647-41f9-94aa-6dbc76a973d4.us-west-1-0.aws.cloud.qdrant.io:6333",
        port = 6333,
        api_key = api_key.QDRANT_API_KEY
    )
    st.session_state.qdrant_db = database.Qdrant_DB(st.session_state.qdrant_client)
    st.session_state.retriever = retriever.create_full_retriever(st.session_state.qdrant_db)
    st.session_state.session_started = True
    st.session_state.bot_memory = memory.memory
    st.session_state.summarized_history = memory.history
    st.session_state.complete_history = []
    st.session_state.langue = "Français"
    st.session_state.previous_retriever_name = ""
    
# display messages
for msg in st.session_state.complete_history :
    with st.chat_message(msg["role"]) :
        st.markdown(msg["content"])

# option buttons (select languages, select retriever and clear the conversation)
st.sidebar.selectbox("Choix de la langue", ["Français", "English", "Deutsch"], key = "language")
st.sidebar.selectbox("Choix du retriever", ["Embedding/BM25 retrievers + rerank", "Embedding retriever", "BM25 retriever"], key = "retriever_name")
st.sidebar.slider("Top-K pour l'Embedding Retriever", min_value=2, max_value=150, value=50, key="top_k_embedding")
st.sidebar.slider("Top-K pour le BM25 Retriever", min_value=2, max_value=30, value=15, key="top_k_bm25")
st.sidebar.slider("Top-N pour le Reranker", min_value=2, max_value=10, value=6, key="top_n_rerank")
show_scores = st.sidebar.checkbox("Afficher les scores de pertinence des sources", value=False)

if st.sidebar.button("Effacer la conversation") :
    st.session_state.bot_memory = memory.memory
    st.session_state.summarized_history = memory.history
    st.session_state.complete_history = []
    st.rerun()

# chat with the LLM
query = st.chat_input("Say something...")
if query is not None and query.strip() != "" :

    with st.chat_message("user") :
        st.markdown(query)

    if (
        st.session_state.previous_retriever_name != st.session_state.retriever_name or
        st.session_state.previous_top_k_embedding != st.session_state.top_k_embedding or
        st.session_state.previous_top_k_bm25 != st.session_state.top_k_bm25 or
        st.session_state.previous_top_n_rerank != st.session_state.top_n_rerank
    ):
        st.session_state.retriever = retriever.create_chosen_retriever(
            st.session_state.qdrant_db, 
            retriever_name=st.session_state.retriever_name, 
            emb_k=st.session_state.top_k_embedding, 
            bm25_k=st.session_state.top_k_bm25, 
            reranker_n=st.session_state.top_n_rerank
        ) 
        
        st.session_state.previous_retriever_name = st.session_state.retriever_name
        st.session_state.previous_top_k_embedding = st.session_state.top_k_embedding
        st.session_state.previous_top_k_bm25 = st.session_state.top_k_bm25
        st.session_state.previous_top_n_rerank = st.session_state.top_n_rerank
    
        st.write(f"Retriever mis à jour : {st.session_state.retriever_name} (Embedding k={st.session_state.top_k_embedding}, BM25 k={st.session_state.top_k_bm25}, Reranker n={st.session_state.top_n_rerank})")

    
    context, sources = retriever.retrieve_context_and_sources(st.session_state.retriever, query, show_scores)
    str_sources = "\n" + "\n".join(f"- {src}" for src in sources)
    prompt = merge(query, st.session_state.summarized_history, context, st.session_state.language)
    time.sleep(1.1)
    
    with st.chat_message("assistant"):
        full_response = ""
        resp_area = st.empty()
        for token in chat(st.session_state.mistral_client, prompt):
            full_response += token
            resp_area.markdown(full_response)
        for token in str_sources :
            full_response += token
            resp_area.markdown(full_response)
            time.sleep(0.02)
        mistral_response = full_response
    
    st.session_state.bot_memory, st.session_state.summarized_history = memory.update_memory(
        st.session_state.bot_memory, 
        ChatMessage(role = "user", content = query), 
        ChatMessage(role = "assistant", content = str(mistral_response))
    )
    
    st.session_state.complete_history.append({"role" : "user", "content" : query})
    st.session_state.complete_history.append({"role" : "assistant", "content" : mistral_response})

Overwriting chatbot.py


In [ ]:
!curl ipv4.icanhazip.com
!streamlit run chatbot.py &>./logs.txt & npx localtunnel --port 8501

35.204.85.145
⠙your url is: https://nice-coins-open.loca.lt
